In [0]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
signs = pd.read_csv('data/signnames.csv')


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
labels_dict = signs.to_dict()['b']

In [0]:
#sieć potrzebuje macierzy 
if y_train.ndim == 1 : y_train=to_categorical(y_train)
if y_test.ndim == 1 : y_test=to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callbacks]
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [10]:
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),
                     
                     Flatten(), 
                     
                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(num_classes, activation='softmax')
  ])



train_and_predict(get_cnn_v6(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.4906 - accuracy: 0.3080 - val_loss: 1.0048 - val_accuracy: 0.6748
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.7655 - accuracy: 0.7545 - val_loss: 0.1769 - val_accuracy: 0.9475
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3340 - accuracy: 0.8958 - val_loss: 0.0726 - val_accuracy: 0.9790
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2272 - accuracy: 0.9319 - val_loss: 0.0437 - val_accuracy: 0.9875
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1672 - accuracy: 0.9509 - val_loss: 0.0364 - val_accuracy: 0.9899


0.9594104308390022

In [13]:
model_trained.evaluate(X_test, y_test)

NameError: ignored

In [0]:
def get_model(params):
  return Sequential([
                    #  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                    #  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  MaxPool2D(),
                    #  Dropout(params['dropout_cnn_block_one']),

                    #  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                    #  MaxPool2D(),
                    #  Dropout(params['dropout_cnn_block_two']),

                    #  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                    #  Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                    #  MaxPool2D(),
                    #  Dropout(params['dropout_cnn_block_three']),
                     
                    #  Flatten(), 
                     
                    #  Dense(1024, activation='relu'),
                    #  Dropout(params['dropout_dense_block_one']),

                    #  Dense(1024, activation='relu'),
                    #  Dropout(params['dropout_dense_block_two']),

                    #  Dense(num_classes, activation='softmax')
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_one']),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_two']),
                     
                     Flatten(), 
                     
                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_one']),

                     Dense(num_classes, activation='softmax')

  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0,
    
      )
  accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
  print (params, 'accuracy: {}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [17]:
space ={
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.438362059801817, 'dropout_cnn_block_three': 0.3927428437896199, 'dropout_cnn_block_two': 0.3327864255241518, 'dropout_dense_block_one': 0.4649257988063085, 'dropout_dense_block_two': 0.6919750160017729}
accuracy: 0.9632652997970581
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.31286906285907756, 'dropout_cnn_block_three': 0.43466631472515493, 'dropout_cnn_block_two': 0.48914483074792214, 'dropout_dense_block_one': 0.6001193049546472, 'dropout_dense_block_two': 0.6411559679865348}
accuracy: 0.9546485543251038
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.378187123212169, 'dropout_cnn_block_three': 0.4816850263520665, 'dropout_cnn_block_two': 0.42434692991868567, 'dropout_dense_block_one': 0.5802262271965699, 'dropout_dense_block_two': 0.6318128475414153}
accuracy: 0.8605442047119141
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4605581090393184, 'dropout_cnn_block_three': 0.3907569768482922, 'dropout_cnn_block_two': 0.32805832411